# TESTING A DIFFERENT SOLVER

In [2]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [1]:
import pandas as pd

cost = pd.read_csv("cost_vector.csv", index_col= 0 )
ebitda = pd.read_csv("ebitda_matrix.csv",index_col= 0)


In [2]:
ebitda = ebitda * 20

In [3]:
import pyomo.environ as pyo

In [6]:
model = pyo.ConcreteModel()

num_y = cost.size

num_rows = ebitda.shape[0]
num_cols = ebitda.shape[1]

# Create x
model.rows = pyo.RangeSet(1,num_rows)
model.cols = pyo.RangeSet(1,num_cols)

# Define decision variables
model.x = pyo.Var(model.rows, model.cols, within = pyo.Binary)
model.y = pyo.Var(range(1, num_y + 1), within = pyo.Binary)

# Define the objective function programmatically
def profit_function(model):
    # Calculate total cost
    cost_total = sum(model.y[j] * cost.iloc[j - 1, 0] for j in range(1, num_y + 1))

    # Calculate total EBITDA
    # ebitda_total = sum(model.y[j] * ebitda.iloc[i - 1, j - 1] * model.x[i, j] for i in model.rows for j in range(1, num_y + 1))
    ebitda_total = sum(model.y[j] * sum(ebitda.iloc[i-1, j-1] * model.x[i,j] for i in range(1, num_rows+1)) for j in range(1,num_y+1))
    # Calculate profit
    profit_total = ebitda_total - cost_total
    return profit_total

model.obj = pyo.Objective(rule = profit_function, sense = pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()
for i in range(1,num_rows):
    model.constraints.add(sum(model.x[i,j] for j in range(1,num_cols)) <= 1)

model.constraints2 = pyo.ConstraintList()
for i in range(1,num_rows):
    for j in range(1,num_cols):
        model.constraints2.add(model.x[i,j] <= model.y[j])


# model.fixed_num_stores = pyo.Constraint(expr = sum(model.y[j]for j in range(1, num_y+1)) == 1)

In [7]:
import os 

os.environ['NEOS_EMAIL'] = 'obamba370@gmail.com'
solver_manager = pyo.SolverManagerFactory('neos')
instance = model.create_instance()
results = solver_manager.solve(instance, opt='cplex', load_solutions=True)
    # ['bonmin', 'cbc', 'conopt', 'couenne', 'cplex', 'filmint', 'filter', 'ipopt', 'knitro', 'l-bfgs-b', 'lancelot', 'lgo', 'loqo', 'minlp', 'minos', 'minto', 'mosek', 'octeract', 'ooqp', 'path', 'raposa', 'snopt']
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 112946
  Sense: unknown
Solver: 
- Status: ok
  Message: CPLEX 22.1.1.0\x3a optimal integer solution; objective 6506809992.201092; 11002 MIP simplex iterations; 0 branch-and-bound nodes; absmipgap = 5.72205e-06, relmipgap = 8.79393e-16; No basis.
  Termination condition: optimal
  Id: 2
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
# solver = pyo.SolverFactory('bonmin')
solver = pyo.SolverFactory('couenne')
results = solver.solve(model, tee = True)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Reading problem: 0.2 seconds
Reformulating problem: 1741.6 seconds
